# Diagnostic 15: All Strategies with Percentage-Based Framework

**Purpose:** Validate all 9 strategies using percentage-based thresholds

**Strategies Tested:**
1. ImmediateSaleStrategy (baseline)
2. EqualBatchStrategy (baseline)
3. PriceThresholdStrategy (baseline)
4. MovingAverageStrategy (baseline)
5. PriceThresholdPredictive (matched pair)
6. MovingAveragePredictive (matched pair)
7. ExpectedValueStrategy (prediction)
8. ConsensusStrategy (prediction)
9. RiskAdjustedStrategy (prediction)

**Key Tests:**
- All strategies use percentage-based logic (scale-invariant)
- Matched pairs mirror baselines exactly
- Prediction strategies beat baselines with 90% accuracy
- No unit mismatch bugs (cents/lb vs dollars)

**Expected Results:**
- Baselines: consistent performance (~$700k-$750k)
- Prediction strategies: beat baselines (+5% to +15%)
- Matched pairs: show value of prediction overlay

In [0]:
%run ../00_setup_and_config

In [0]:
import sys
import os
import pandas as pd
import numpy as np
import pickle
from datetime import datetime, timedelta
import importlib.util

print("="*80)
print("DIAGNOSTIC 15: ALL STRATEGIES - PERCENTAGE-BASED FRAMEWORK")
print("="*80)
print("\nGoal: Validate all 9 strategies with percentage-based logic")

## Load All Strategies

In [ ]:
# Load all strategies from all_strategies_pct.py
# Force fresh reload to pick up latest changes
import sys
if 'all_strategies_pct' in sys.modules:
    del sys.modules['all_strategies_pct']

spec = importlib.util.spec_from_file_location("all_strategies_pct", "all_strategies_pct.py")
strategies_module = importlib.util.module_from_spec(spec)
spec.loader.exec_module(strategies_module)

ImmediateSaleStrategy = strategies_module.ImmediateSaleStrategy
EqualBatchStrategy = strategies_module.EqualBatchStrategy
PriceThresholdStrategy = strategies_module.PriceThresholdStrategy
MovingAverageStrategy = strategies_module.MovingAverageStrategy
PriceThresholdPredictive = strategies_module.PriceThresholdPredictive
MovingAveragePredictive = strategies_module.MovingAveragePredictive
ExpectedValueStrategy = strategies_module.ExpectedValueStrategy
ConsensusStrategy = strategies_module.ConsensusStrategy
RiskAdjustedStrategy = strategies_module.RiskAdjustedStrategy

print("✓ Loaded all 9 percentage-based strategies from all_strategies_pct.py (fresh reload)")

# Verify the fix is loaded
print(f"✓ PriceThresholdStrategy.last_sale_day init: {PriceThresholdStrategy().last_sale_day} (should be 0)")
print(f"✓ MovingAverageStrategy.last_sale_day init: {MovingAverageStrategy().last_sale_day} (should be 0)")

## Load Data

In [0]:
COMMODITY = 'coffee'
MODEL_VERSION = 'synthetic_acc90'

DATA_PATHS = get_data_paths(COMMODITY, MODEL_VERSION)
COMMODITY_CONFIG = COMMODITY_CONFIGS[COMMODITY]

print("Loading prices...")
prices_table = get_data_paths(COMMODITY)['prices_prepared']
prices = spark.table(prices_table).toPandas()
prices['date'] = pd.to_datetime(prices['date'])
print(f"✓ Loaded {len(prices)} price records")

print("\nLoading predictions...")
matrices_path = DATA_PATHS['prediction_matrices']
with open(matrices_path, 'rb') as f:
    prediction_matrices = pickle.load(f)
print(f"✓ Loaded {len(prediction_matrices)} prediction matrices")

# Normalize dates
prediction_matrices = {pd.to_datetime(k): v for k, v in prediction_matrices.items()}

print(f"\nCommodity: {COMMODITY}")
print(f"Storage cost: {COMMODITY_CONFIG['storage_cost_pct_per_day']}% per day")
print(f"Transaction cost: {COMMODITY_CONFIG['transaction_cost_pct']}%")

In [ ]:
# Override with small farmer realistic costs
from cost_config_small_farmer import get_small_farmer_config

COMMODITY_CONFIG = get_small_farmer_config(COMMODITY_CONFIG)

print("\n" + "="*80)
print("UPDATED COSTS - SMALL FARMER REALITY")
print("="*80)
print(f"Storage cost: {COMMODITY_CONFIG['storage_cost_pct_per_day']}% per day (was 0.025%)")
print(f"Transaction cost: {COMMODITY_CONFIG['transaction_cost_pct']}% per sale (was 0.25%)")
print("\nBased on research:")
print("  - Storage: Quality degradation only (on-farm gunny sack storage)")
print("  - Transaction: Local intermediary fee (immediate cash payment)")
print("  - This is 5x lower storage cost, 25x lower transaction cost")


## Create Backtest Engine

In [0]:
class DiagnosticBacktestEngine:
    """
    Simplified backtest engine for diagnostic testing.
    """
    
    def __init__(self, prices_df, prediction_matrices, commodity_config):
        self.prices = prices_df
        self.prediction_matrices = prediction_matrices
        self.config = commodity_config
        
    def run_backtest(self, strategy, initial_inventory=50.0):
        """Run backtest for a single strategy"""
        
        inventory = initial_inventory
        trades = []
        total_revenue = 0
        total_transaction_costs = 0
        total_storage_costs = 0
        
        strategy.reset()
        strategy.set_harvest_start(0)
        
        for day in range(len(self.prices)):
            current_date = self.prices.iloc[day]['date']
            current_price = self.prices.iloc[day]['price']
            
            price_history = self.prices.iloc[:day+1].copy()
            predictions = self.prediction_matrices.get(current_date, None)
            
            decision = strategy.decide(
                day=day,
                inventory=inventory,
                current_price=current_price,
                price_history=price_history,
                predictions=predictions
            )
            
            if decision['action'] == 'SELL' and decision['amount'] > 0:
                amount = min(decision['amount'], inventory)
                
                price_per_ton = current_price * 20
                revenue = amount * price_per_ton
                transaction_cost = revenue * (self.config['transaction_cost_pct'] / 100)
                
                total_revenue += revenue
                total_transaction_costs += transaction_cost
                inventory -= amount
                
                trades.append({
                    'day': day,
                    'date': current_date,
                    'amount': amount,
                    'price': current_price,
                    'revenue': revenue,
                    'transaction_cost': transaction_cost,
                    'reason': decision['reason']
                })
            
            if inventory > 0:
                avg_price = self.prices.iloc[:day+1]['price'].mean()
                price_per_ton = avg_price * 20
                storage_cost = inventory * price_per_ton * (self.config['storage_cost_pct_per_day'] / 100)
                total_storage_costs += storage_cost
        
        net_earnings = total_revenue - total_transaction_costs - total_storage_costs
        
        return {
            'strategy': strategy.name,
            'net_earnings': net_earnings,
            'total_revenue': total_revenue,
            'total_transaction_costs': total_transaction_costs,
            'total_storage_costs': total_storage_costs,
            'num_trades': len(trades),
            'final_inventory': inventory,
            'trades': trades
        }

engine = DiagnosticBacktestEngine(prices, prediction_matrices, COMMODITY_CONFIG)
print("✓ Diagnostic backtest engine created")

## Test All 9 Strategies

In [0]:
print("="*80)
print("TESTING ALL 9 PERCENTAGE-BASED STRATEGIES")
print("="*80)

results = {}

# Create strategy instances
strategies = [
    # Baselines
    ImmediateSaleStrategy(),
    EqualBatchStrategy(),
    PriceThresholdStrategy(),
    MovingAverageStrategy(),
    
    # Matched pairs
    PriceThresholdPredictive(
        storage_cost_pct_per_day=COMMODITY_CONFIG['storage_cost_pct_per_day'],
        transaction_cost_pct=COMMODITY_CONFIG['transaction_cost_pct']
    ),
    MovingAveragePredictive(
        storage_cost_pct_per_day=COMMODITY_CONFIG['storage_cost_pct_per_day'],
        transaction_cost_pct=COMMODITY_CONFIG['transaction_cost_pct']
    ),
    
    # Standalone predictions
    ExpectedValueStrategy(
        storage_cost_pct_per_day=COMMODITY_CONFIG['storage_cost_pct_per_day'],
        transaction_cost_pct=COMMODITY_CONFIG['transaction_cost_pct']
    ),
    ConsensusStrategy(
        storage_cost_pct_per_day=COMMODITY_CONFIG['storage_cost_pct_per_day'],
        transaction_cost_pct=COMMODITY_CONFIG['transaction_cost_pct']
    ),
    RiskAdjustedStrategy(
        storage_cost_pct_per_day=COMMODITY_CONFIG['storage_cost_pct_per_day'],
        transaction_cost_pct=COMMODITY_CONFIG['transaction_cost_pct']
    ),
]

print(f"\nRunning backtests for {len(strategies)} strategies...\n")

for strategy in strategies:
    print(f"Testing {strategy.name}...")
    result = engine.run_backtest(strategy)
    results[strategy.name] = result
    
    print(f"  Net earnings: ${result['net_earnings']:,.2f}")
    print(f"  Trades: {result['num_trades']}")
    print(f"  Storage costs: ${result['total_storage_costs']:,.2f}")
    print(f"  Final inventory: {result['final_inventory']:.1f} tons")
    print()

print("✓ All backtests complete")

## Performance Comparison

In [0]:
print("="*80)
print("PERFORMANCE RANKING")
print("="*80)

# Sort by net earnings
ranked = sorted(results.items(), key=lambda x: x[1]['net_earnings'], reverse=True)

best_baseline = max([
    results['Immediate Sale'],
    results['Equal Batches'],
    results['Price Threshold'],
    results['Moving Average']
], key=lambda x: x['net_earnings'])

best_baseline_earnings = best_baseline['net_earnings']
best_baseline_name = best_baseline['strategy']

print(f"\nBest Baseline: {best_baseline_name} (${best_baseline_earnings:,.2f})\n")
print("Rank | Strategy                    | Net Earnings  | vs Best Baseline | Trades | Storage")
print("-" * 95)

for i, (name, result) in enumerate(ranked, 1):
    vs_baseline = 100 * (result['net_earnings'] - best_baseline_earnings) / best_baseline_earnings
    print(f"{i:4d} | {name:27s} | ${result['net_earnings']:12,.2f} | {vs_baseline:+7.2f}% | {result['num_trades']:6d} | ${result['total_storage_costs']:7,.0f}")

# Identify winners
print("\n" + "="*80)
print("BEATING BASELINE")
print("="*80)

winners = [(name, result) for name, result in results.items() 
           if result['net_earnings'] > best_baseline_earnings]

if winners:
    print(f"\n✓ {len(winners)} strategies beat the best baseline:\n")
    for name, result in sorted(winners, key=lambda x: x[1]['net_earnings'], reverse=True):
        improvement = 100 * (result['net_earnings'] - best_baseline_earnings) / best_baseline_earnings
        dollar_gain = result['net_earnings'] - best_baseline_earnings
        print(f"  {name}: ${result['net_earnings']:,.2f} (+{improvement:.2f}%, +${dollar_gain:,.2f})")
else:
    print("\n⚠️  No strategies beat the best baseline")
    print("    Further optimization needed")

## Matched Pair Analysis

In [0]:
print("="*80)
print("MATCHED PAIR COMPARISON")
print("="*80)

pairs = [
    ('Price Threshold', 'Price Threshold Predictive'),
    ('Moving Average', 'Moving Average Predictive')
]

for baseline_name, predictive_name in pairs:
    baseline = results[baseline_name]
    predictive = results[predictive_name]
    
    improvement = 100 * (predictive['net_earnings'] - baseline['net_earnings']) / baseline['net_earnings']
    dollar_gain = predictive['net_earnings'] - baseline['net_earnings']
    
    print(f"\n{baseline_name} vs {predictive_name}:")
    print(f"  Baseline:   ${baseline['net_earnings']:,.2f}")
    print(f"  Predictive: ${predictive['net_earnings']:,.2f}")
    print(f"  Improvement: {improvement:+.2f}% (${dollar_gain:+,.2f})")
    
    if improvement > 0:
        print(f"  ✓ Predictions add value!")
    else:
        print(f"  ⚠️  Predictions reduce performance")

## Save Results

In [0]:
import json

summary = {
    'timestamp': datetime.now().isoformat(),
    'commodity': COMMODITY,
    'model_version': MODEL_VERSION,
    'best_baseline': {
        'name': best_baseline_name,
        'earnings': float(best_baseline_earnings)
    },
    'strategies': {}
}

for name, result in results.items():
    vs_baseline = 100 * (result['net_earnings'] - best_baseline_earnings) / best_baseline_earnings
    summary['strategies'][name] = {
        'earnings': float(result['net_earnings']),
        'vs_baseline_pct': float(vs_baseline),
        'trades': result['num_trades'],
        'storage_costs': float(result['total_storage_costs']),
        'final_inventory': float(result['final_inventory'])
    }

# Save to Volume
json_path = '/Volumes/commodity/trading_agent/files/diagnostic_15_results.json'
with open(json_path, 'w') as f:
    json.dump(summary, f, indent=2)

print(f"✓ Results saved to: {json_path}")

# Also save pickle with full details
pkl_path = '/Volumes/commodity/trading_agent/files/diagnostic_15_results.pkl'
with open(pkl_path, 'wb') as f:
    pickle.dump(results, f)

print(f"✓ Detailed results saved to: {pkl_path}")

print("\n" + "="*80)
print("DIAGNOSTIC 15 COMPLETE")
print("="*80)